<a href="https://colab.research.google.com/github/IshanKapadia-Data/quant-screener-ai-trader/blob/main/screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import yfinance as yf
from time import sleep
from tqdm import tqdm
from datetime import datetime

# Step 1: Load S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    table = pd.read_html(url)
    df = table[0]
    tickers = df['Symbol'].tolist()
    return tickers

# Get tickers and fix symbols for yfinance
tickers = get_sp500_tickers()
tickers = [ticker.replace('.', '-') for ticker in tickers]

In [2]:
# Step 2: RSI Calculation
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [3]:
# Step 3: Fetch stock data
results = []

print("Fetching stock data...")

for ticker in tqdm(tickers):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period='2mo', interval='1d')

        if len(hist) < 15 or hist['Close'].isnull().all():
            continue

        rsi_series = calculate_rsi(hist['Close'])
        latest_rsi = rsi_series.dropna().iloc[-1]

        info = stock.info
        current_price = info.get('regularMarketPrice')
        fifty_two_week_high = info.get('fiftyTwoWeekHigh')
        pe_ratio = info.get('trailingPE')

        if current_price and fifty_two_week_high:
            pct_below_high = ((fifty_two_week_high - current_price) / fifty_two_week_high) * 100
            if pct_below_high <= 10:
                results.append({
                    'Ticker': ticker,
                    'Current Price': current_price,
                    '52-Week High': fifty_two_week_high,
                    '% Below High': round(pct_below_high, 2),
                    'P/E Ratio': pe_ratio,
                    'RSI': round(latest_rsi, 2)
                })

        sleep(0.5)
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue


Fetching stock data...


100%|██████████| 503/503 [09:28<00:00,  1.13s/it]


In [4]:
# Step 4: Create and display DataFrame
df_result = pd.DataFrame(results)
df_result = df_result.sort_values(by='% Below High')

pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:,.2f}'.format)
display(df_result)

,Ticker,Current Price,52-Week High,% Below High,P/E Ratio,RSI
128,NOC,581.62,582.16,0.09,21.46,83.89
65,EVRG,71.50,71.63,0.18,18.82,67.99
7,MO,62.50,62.65,0.23,12.09,75.16
184,WEC,110.67,111.07,0.36,21.24,67.58
31,CBOE,248.68,249.63,0.38,32.63,67.10
191,XEL,73.82,74.13,0.42,20.45,71.67
125,NI,42.66,42.90,0.55,23.06,75.94
34,CNP,39.08,39.31,0.60,27.33,71.69
2,ALLE,165.32,166.37,0.63,22.96,74.29
147,RMD,278.82,280.86,0.72,29.29,78.24


In [5]:
# Step 5: Save to CSV with dynamic date
today_str = datetime.today().strftime("%Y-%m-%d")
filename = f"sp500_watchlist_{today_str}.csv"
df_result.to_csv(filename, index=False)